### Age prediction model

DSEMT group 3 Final Project Part 1:
                                  - Navin Sharma 002737702
                                  - Shreya Deshpande 001002875
                                  - Koushal Sai Amshala 002772702
                                  - Parjita Munshi 002761776

In [ ]:
import pandas as pd
import tensorflow as tf
import glob
import os
from google.colab import drive
import matplotlib.pyplot as plt

#Fetching the data from goolge drive
drive.mount('/content/gdrive/')
!unzip gdrive/MyDrive/archive.zip
!unzip gdrive/MyDrive/Train.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train/15256.jpg         
  inflating: __MACOSX/Train/._15256.jpg  
  inflating: Train/8422.jpg          
  inflating: __MACOSX/Train/._8422.jpg  
  inflating: Train/1374.jpg          
  inflating: __MACOSX/Train/._1374.jpg  
  inflating: Train/9728.jpg          
  inflating: __MACOSX/Train/._9728.jpg  
  inflating: Train/78.jpg            
  inflating: __MACOSX/Train/._78.jpg  
  inflating: Train/8436.jpg          
  inflating: __MACOSX/Train/._8436.jpg  
  inflating: Train/15242.jpg         
  inflating: __MACOSX/Train/._15242.jpg  
  inflating: Train/838.jpg           
  inflating: __MACOSX/Train/._838.jpg  
  inflating: Train/25381.jpg         
  inflating: __MACOSX/Train/._25381.jpg  
  inflating: Train/11024.jpg         
  inflating: __MACOSX/Train/._11024.jpg  
  inflating: Train/3563.jpg          
  inflating: __MACOSX/Train/._3563.jpg  
  inflating: Train/19478.jpg         
  inflating: __MACOSX/Train/._19478.jpg  

In [ ]:
train_data = pd.read_csv('/content/archive/train.csv')
train_data.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [ ]:
#classifying young middle and old in numerical format
train_data['Class'].unique()

train_data['Class'].replace(['YOUNG', 'MIDDLE','OLD'],[0, 1, 2], inplace=True)
train_data.head()

train_data.Class.unique()

array([1, 0, 2])

In [ ]:
def trainimage(path,ch = 3, resize=(512,512)):

	kp = tf.io.read_file(path)
	kp = tf.image.decode_jpeg(kp, channels=ch)
	kp = tf.image.convert_image_dtype(kp, dtype=tf.float32)
	kp = tf.image.resize(kp, resize)
	return kp

def load_data(image_path, label):
    image = trainimage(image_path, 3, (512,512))
    
    return (image, label)

In [ ]:
train_image_paths = glob.glob('/content/Train/*.jpg')
print(len(train_image_paths))

label_list = []
for i in train_image_paths:
    _,tail = os.path.split(i)
    label = train_data.loc[train_data['ID'] == tail]['Class'].values[0]
    label_list.append(label)
print(len(label_list))

19906
19906


In [ ]:
# Train Test Split
train_size = int(0.9*(len(train_image_paths)))
test_size = int(0.1*(len(train_image_paths)))

traindataset = tf.data.Dataset.from_tensor_slices((train_image_paths[:train_size], label_list[:train_size]))
testdataset = tf.data.Dataset.from_tensor_slices((train_image_paths[train_size:], label_list[train_size:]))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
traindataset = (traindataset.map(load_data, num_parallel_calls=AUTOTUNE).batch(64).prefetch(AUTOTUNE))
print(traindataset)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
testdataset = (testdataset.map(load_data, num_parallel_calls=AUTOTUNE).batch(64).prefetch(AUTOTUNE))
print(testdataset)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
preprocess_model = tf.keras.applications.MobileNetV2(input_shape = (512,512,3,),include_top=False,weights='imagenet')
preprocess_model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 256, 256, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 256, 256, 32  128         ['Conv1[0][0]']                  
                

In [ ]:
preprocess_model.trainable = False

inputs = tf.keras.Input(shape=(512, 512, 3))
x = preprocess_input(inputs)
x = preprocess_model(x, training=True)
x = tf.keras.layers.GlobalMaxPooling2D()(x)
x = tf.keras.layers.Dropout(0.35)(x)
outputs = tf.keras.layers.Dense(3)(x)
preprocess_model = tf.keras.Model(inputs, outputs)
preprocess_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 512, 512, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 512, 512, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 16, 16, 1280)     2257984   
 ional)                                                          
                                                                 
 global_max_pooling2d (Globa  (None, 1280)             0         
 lMaxPooling2D)                                              

In [ ]:
preprocess_model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
model_history = preprocess_model.fit(traindataset,epochs=1,validation_data=traindataset)

280/280 [==============================] - 6039s 22s/step - loss: 1.6642 - accuracy: 0.4377 - val_loss: 0.9973 - val_accuracy: 0.5493


In [ ]:
print(model_history.history['accuracy'])
print(model_history.history['val_accuracy'])

[0.4377337396144867]
[0.5493162274360657]


In [ ]:
hist = pd.DataFrame(model_history.history)
hist['epoch'] = model_history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
0,1.664182,0.437734,0.997321,0.549316,0


In [ ]:
def plot_loss(model_history):
  print(model_history.history['val_loss'])
  plt.plot([0.47, 0.51], label='loss')
  plt.plot([0.548,0.548], label='val_loss')
  # plt.ylim([0, 3])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [ ]:
pred = preprocess_model.predict(testdataset)

32/32 [==============================] - 330s 10s/step


In [ ]:
print(pred)

[[ 1.011444   1.1405895 -1.0824145]
 [ 1.0077229  1.0354536 -1.083391 ]
 [ 1.0849257  1.0751446 -1.1287332]
 ...
 [ 1.0103511  1.08293   -1.1027639]
 [ 1.0256177  1.1078937 -1.1057589]
 [ 1.0562581  1.0490756 -1.0747346]]
